# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token=Tpk_059b97af715d417d9f49f50b51b1c448'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 98220629, 'calculationPrice': 'iexlasttrade', 'change': 1.44, 'changePercent': 0.00921, 'close': 0, 'closeSource': 'lofiifac', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 162, 'iexCloseTime': 1692524675137, 'iexLastUpdated': 1710473203991, 'iexMarketPercent': 0.018499055853726246, 'iexOpen': 163.23, 'iexOpenTime': 1702696486802, 'iexRealtimePrice': 165.19, 'iexRealtimeSize': 103, 'iexVolume': 1369964, 'lastTradeTime': 1663546771492, 'latestPrice': 164, 'latestSource': 'IEX Last Trade', 'latestTime': 'March 17, 2022', 'latestUpdate': 1727960998068, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2723858245128, 'oddLotDelayedPrice': Non

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

27.09

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:2]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token=Tpk_059b97af715d417d9f49f50b51b1c448'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns
        
        ), 
            ignore_index = True
        )
final_dataframe
       

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,141.80,36.43,N/A
1,AAL,17.16,-5.5,N/A
2,AAP,210.73,22.6,N/A
3,AAPL,162.00,27.28,N/A
4,ABBV,163.54,25.68,N/A
...,...,...,...,...
195,FTNT,310.21,85.2,N/A
196,FTV,64.00,37.61,N/A
197,GD,239.73,19.98,N/A
198,GE,96.26,-16.29,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [8]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,57.66,2.83,N/A
1,F,16.95,3.86,N/A
2,BIO,591.37,4.21,N/A
3,COF,142.03,5.17,N/A
4,C,59.90,5.83,N/A
5,AIG,61.58,5.9,N/A
6,DFS,118.29,6.46,N/A
7,DD,79.54,6.59,N/A
8,DHI,84.90,6.86,N/A
9,DOW,63.30,7.47,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [10]:
portfolio_input()

Enter the value of your portfolio:500


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [11]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,57.66,2.83,0
1,F,16.95,3.86,0
2,BIO,591.37,4.21,0
3,COF,142.03,5.17,0
4,C,59.90,5.83,0
5,AIG,61.58,5.9,0
6,DFS,118.29,6.46,0
7,DD,79.54,6.59,0
8,DHI,84.90,6.86,0
9,DOW,63.30,7.47,0


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [12]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token=Tpk_059b97af715d417d9f49f50b51b1c448'
data = requests.get(batch_api_call_url).json()
print(data['AAPL']['advanced-stats'])

#Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']
                        
#Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']
                        
#Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']
                        
#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
print(enterprise_value)
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda
                        
#Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit                      

{'beta': 1.3052810834437554, 'totalCash': 66854754059, 'currentDebt': 151316869680, 'revenue': 380003272985, 'grossProfit': 170177334760, 'totalRevenue': 385083591743, 'EBITDA': 134024845116, 'revenuePerShare': 23.58, 'revenuePerEmployee': 2601271.71, 'debtToEquity': 5.325809254747293, 'profitMargin': 0.2725156710636422, 'enterpriseValue': 2742213557052, 'enterpriseValueToRevenue': 7.07, 'priceToSales': 7.12, 'priceToBook': 36.26, 'forwardPERatio': 26.003828347771663, 'pegRatio': 0.4201507144036075, 'peHigh': 28.138677974998117, 'peLow': 14.970947407042889, 'week52highDate': '2021-12-26', 'week52lowDate': '2021-03-21', 'putCallRatio': 0.5140543651550665, 'companyName': 'Apple Inc', 'marketcap': 2716798268165, 'week52high': 185.85, 'week52low': 120.81, 'week52highSplitAdjustOnly': 191.72, 'week52highDateSplitAdjustOnly': '2021-12-22', 'week52lowSplitAdjustOnly': 123.77, 'week52lowDateSplitAdjustOnly': '2021-03-23', 'week52change': 0.2874392851446126, 'sharesOutstanding': 17094949053, 'f

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [13]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings[:2]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token=Tpk_059b97af715d417d9f49f50b51b1c448'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try: 
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try: 
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
            
        ],
        index = rv_columns),
            ignore_index = True
        )

In [14]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,141.43,N/A,34.96,N/A,7.89,N/A,6.46,N/A,24.160758,N/A,12.183274,N/A,N/A
1,AAL,16.97,N/A,-5.44,N/A,-1.49,N/A,0.364,N/A,-12.301995,N/A,1.233875,N/A,N/A
2,AAP,218.06,N/A,22.3,N/A,4.05,N/A,1.14,N/A,12.243574,N/A,2.739863,N/A,N/A
3,AAPL,166.00,N/A,27.57,N/A,36.95,N/A,6.99,N/A,20.417836,N/A,16.135287,N/A,N/A
4,ABBV,161.66,N/A,25.16,N/A,18.52,N/A,4.99,N/A,12.541778,N/A,9.138132,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,FTNT,316.72,N/A,84.9,N/A,63.06,N/A,14.65,N/A,61.125935,N/A,17.601295,N/A,N/A
196,FTV,64.00,N/A,38.22,N/A,2.28,N/A,4.22,N/A,20.475959,N/A,8.052725,N/A,N/A
197,GD,235.60,N/A,20.72,N/A,3.64,N/A,1.7,N/A,14.535734,N/A,11.182507,N/A,N/A
198,GE,98.23,N/A,-16.15,N/A,2.7,N/A,1.5,N/A,16.225685,N/A,5.465797,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,188.40,N/A,62.11,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,329.38,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,353.68,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,9.93,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,25.88,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.70,N/A,14.74,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.57,N/A,32.55,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,38.73,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
193,FRT,121.95,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [16]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [17]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [18]:
from scipy.stats import percentileofscore as score
metrics = {
    'Price-to-Earnings Ratio' : 'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
       rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row,metric])/100
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,141.43,N/A,34.96,0.745,7.89,0.79,6.460,0.815,24.160758,0.8,12.183274,0.825,N/A
1,AAL,16.97,N/A,-5.44,0.04,-1.49,0.025,0.364,0.015,-12.301995,0.015,1.233875,0.04,N/A
2,AAP,218.06,N/A,22.30,0.445,4.05,0.54,1.140,0.1375,12.243574,0.34,2.739863,0.13,N/A
3,AAPL,166.00,N/A,27.57,0.6,36.95,0.975,6.990,0.8425,20.417836,0.7,16.135287,0.895,N/A
4,ABBV,161.66,N/A,25.16,0.56,18.52,0.935,4.990,0.715,12.541778,0.355,9.138132,0.675,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,FTNT,316.72,N/A,84.90,0.945,63.06,0.99,14.650,0.965,61.125935,0.99,17.601295,0.915,N/A
196,FTV,64.00,N/A,38.22,0.785,2.28,0.285,4.220,0.605,20.475959,0.71,8.052725,0.53,N/A
197,GD,235.60,N/A,20.72,0.405,3.64,0.465,1.700,0.2475,14.535734,0.485,11.182507,0.765,N/A
198,GE,98.23,N/A,-16.15,0.03,2.70,0.345,1.500,0.2125,16.225685,0.55,5.465797,0.35,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [19]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,141.43,N/A,34.96,0.745,7.89,0.79,6.460,0.815,24.160758,0.8,12.183274,0.825,0.795
1,AAL,16.97,N/A,-5.44,0.04,-1.49,0.025,0.364,0.015,-12.301995,0.015,1.233875,0.04,0.027
2,AAP,218.06,N/A,22.30,0.445,4.05,0.54,1.140,0.1375,12.243574,0.34,2.739863,0.13,0.3185
3,AAPL,166.00,N/A,27.57,0.6,36.95,0.975,6.990,0.8425,20.417836,0.7,16.135287,0.895,0.8025
4,ABBV,161.66,N/A,25.16,0.56,18.52,0.935,4.990,0.715,12.541778,0.355,9.138132,0.675,0.648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,FTNT,316.72,N/A,84.90,0.945,63.06,0.99,14.650,0.965,61.125935,0.99,17.601295,0.915,0.961
196,FTV,64.00,N/A,38.22,0.785,2.28,0.285,4.220,0.605,20.475959,0.71,8.052725,0.53,0.583
197,GD,235.60,N/A,20.72,0.405,3.64,0.465,1.700,0.2475,14.535734,0.485,11.182507,0.765,0.4735
198,GE,98.23,N/A,-16.15,0.03,2.70,0.345,1.500,0.2125,16.225685,0.55,5.465797,0.35,0.2975


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [20]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,16.97,N/A,-5.44,0.04,-1.4900,0.025,0.3640,0.015,-12.301995,0.015,1.233875,0.04,0.027
1,AIG,63.48,N/A,5.71,0.075,0.7923,0.035,0.9918,0.11,3.531705,0.04,0.952015,0.02,0.056
2,F,16.73,N/A,3.91,0.06,1.3700,0.1,0.4938,0.045,4.238742,0.055,2.312407,0.095,0.071
3,C,59.67,N/A,5.92,0.08,0.6381,0.03,1.4400,0.195,3.608435,0.045,1.341760,0.05,0.08
4,DXC,32.96,N/A,-14.26,0.035,1.6900,0.155,0.4870,0.035,4.957009,0.075,2.902414,0.145,0.089
5,AIZ,177.54,N/A,8.09,0.115,1.8300,0.1875,0.9767,0.105,2.145502,0.03,0.971285,0.025,0.0925
6,COF,143.10,N/A,5.23,0.07,0.9803,0.045,1.8600,0.28,3.788398,0.05,1.743662,0.065,0.102
7,BEN,29.15,N/A,7.74,0.11,1.2300,0.0725,1.6800,0.2375,5.479302,0.1,1.453765,0.055,0.115
8,APA,39.46,N/A,15.34,0.255,-8.3400,0.01,1.5600,0.22,4.677344,0.065,2.425140,0.105,0.131
9,DHI,83.60,N/A,6.68,0.095,1.9000,0.21,1.0000,0.115,5.265670,0.09,3.551600,0.17,0.136


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [21]:
portfolio_input()

Enter the value of your portfolio:500


In [26]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = position_size/rv_dataframe.loc[row, 'Price']

rv_dataframe

C:\Users\conno\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,16.97,0.589275,-5.44,0.04,-1.4900,0.025,0.3640,0.015,-12.301995,0.015,1.233875,0.04,0.027
1,AIG,63.48,0.15753,5.71,0.075,0.7923,0.035,0.9918,0.11,3.531705,0.04,0.952015,0.02,0.056
2,F,16.73,0.597729,3.91,0.06,1.3700,0.1,0.4938,0.045,4.238742,0.055,2.312407,0.095,0.071
3,C,59.67,0.167588,5.92,0.08,0.6381,0.03,1.4400,0.195,3.608435,0.045,1.341760,0.05,0.08
4,DXC,32.96,0.303398,-14.26,0.035,1.6900,0.155,0.4870,0.035,4.957009,0.075,2.902414,0.145,0.089
5,AIZ,177.54,0.056325,8.09,0.115,1.8300,0.1875,0.9767,0.105,2.145502,0.03,0.971285,0.025,0.0925
6,COF,143.10,0.069881,5.23,0.07,0.9803,0.045,1.8600,0.28,3.788398,0.05,1.743662,0.065,0.102
7,BEN,29.15,0.343053,7.74,0.11,1.2300,0.0725,1.6800,0.2375,5.479302,0.1,1.453765,0.055,0.115
8,APA,39.46,0.253421,15.34,0.255,-8.3400,0.01,1.5600,0.22,4.677344,0.065,2.425140,0.105,0.131
9,DHI,83.60,0.119617,6.68,0.095,1.9000,0.21,1.0000,0.115,5.265670,0.09,3.551600,0.17,0.136


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [23]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)


## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = {

                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': [ 'PB Percentile', percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }
for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
writer.save()

## Saving Our Excel Output
As before, saving our Excel output is very easy: